In [1]:
import xlsxwriter
import urllib
from bs4 import BeautifulSoup
#Ensure you update this

match_link="https://www.espncricinfo.com/series/8048/scorecard/1216492/mumbai-indians-vs-chennai-super-kings-1st-match-indian-premier-league-2020-21"
match_no=1
innings=['MI','CSK']
man_of_the_match="A%Rayudu%"
winning_team="CSK"
#duck_out_in0ball_players_list=[""]
#not_out_in_0_players_list=[""]

In [2]:

teams="1	MI vs CSK	Govardhan	Rohit Sharma	Kieron Pollard✈	Saurabh Tiwary	Ambati Rayudu	Dwayne Bravo✈	Ravindra Jadeja	Ishan Kishan	James Pattinson✈	Lungisani Ngidi✈	Rahul Chahar	Jasprit Bumrah	Saurabh Tiwary	Kieron Pollard✈	MI	\
1	MI vs CSK	Deepak	Surya Kumar Yadav	Rohit Sharma	Ambati Rayudu	Kieron Pollard✈	Ravindra Jadeja	Hardik Pandya	Quinton De Kock✈	Deepak Chahar	Imran Tahir✈	Jasprit Bumrah	Rahul Chahar	Hardik Pandya	Hardik Pandya	MI	\
1	MI vs CSK	Suresh	Ambati Rayudu	Shane Watson✈	Quinton De Kock✈	Rohit Sharma	Kieron Pollard✈	Hardik Pandya	MS Dhoni	Deepak Chahar	Ravindra Jadeja	Jasprit Bumrah	Rahul Chahar	Quinton De Kock✈	Rahul Chahar	MI	\
1	MI vs CSK	Kiran	MS Dhoni	Rohit Sharma	Surya Kumar Yadav	Shane Watson✈	Hardik Pandya	Ravindra Jadeja	Quinton De Kock✈	Jasprit Bumrah	Kieron Pollard✈	Imran Tahir✈	Deepak Chahar	Deepak Chahar	Rohit Sharma	MI	\
1	MI vs CSK	Mitesh	Rohit Sharma	Surya Kumar Yadav	Ambati Rayudu	Kieron Pollard✈	Hardik Pandya	Ravindra Jadeja	Quinton De Kock✈	Dwayne Bravo✈	Jasprit Bumrah	Deepak Chahar	Krunal Pandya	Kieron Pollard✈	Dwayne Bravo✈	CSK	\
1	MI vs CSK	Sowmya	Rohit Sharma	Shane Watson✈	Kieron Pollard✈	Surya Kumar Yadav	Hardik Pandya	Ravindra Jadeja	MS Dhoni	Jasprit Bumrah	Imran Tahir✈	Krunal Pandya	Dwayne Bravo✈	MS Dhoni	Jasprit Bumrah	MI	\
1	MI vs CSK	Aditya	Faf Du Plessis✈	Shane Watson✈	Surya Kumar Yadav	Rohit Sharma	Ravindra Jadeja	Hardik Pandya	MS Dhoni	Mitchell Santner✈	Jasprit Bumrah	Deepak Chahar	Trent Boult✈	MS Dhoni	Shane Watson✈	CSK	\
1	MI vs CSK	Shriyansh	Rohit Sharma	Quinton De Kock✈	Surya Kumar Yadav	Faf Du Plessis✈	Hardik Pandya	Shane Watson✈	MS Dhoni	Krunal Pandya	Deepak Chahar	Ravindra Jadeja	Jasprit Bumrah	Hardik Pandya	Ravindra Jadeja	CSK	\
1	MI vs CSK	Avani	Rohit Sharma	Faf Du Plessis✈	Quinton De Kock✈	Surya Kumar Yadav	Dwayne Bravo✈	Hardik Pandya	MS Dhoni	Jasprit Bumrah	Imran Tahir✈	Deepak Chahar	Ravindra Jadeja	MS Dhoni	Rohit Sharma	MI	"
teams=teams.replace("✈","")

In [3]:
#Connect to cricinfo website and get the response into soup using beautiful soup library
import pandas as pd
import html5lib
sock = urllib.request.urlopen(match_link) 
htmlSource = sock.read()                  
soup = BeautifulSoup(htmlSource, 'html.parser')
batsmen_data=soup.find_all("table",{"class":"table batsman"})
bowlers_data=soup.find_all("div",{"class":"scorecard-section bowling"})
all_queries_list=[]


#	Player_Name	is_out	R	B	4s	6s	SR	Team	4s_points	6s_points	runs_points	runs_bonus_points	sr_points	total_batting_points

#extract 1st innings batting score
dfbat1 = pd.read_html(match_link,skiprows=0)[0]
dfbat1['Team']=innings[0]
dfbat1['Team_Field']=innings[1]



#extract 2nd innings batting score
dfbat2 = pd.read_html(match_link,skiprows=0)[2]
dfbat2['Team']=innings[1]
dfbat2['Team_Field']=innings[0]

#combine 2 innings batting score
final_batting_scorecard=pd.concat([dfbat1, dfbat2])

#Remove Unnecessary rows
final_batting_scorecard = final_batting_scorecard[final_batting_scorecard['BATSMEN'].notna()]
final_batting_scorecard=final_batting_scorecard[final_batting_scorecard.BATSMEN != 'TOTAL']
final_batting_scorecard=final_batting_scorecard[final_batting_scorecard.BATSMEN != 'Extras']
final_batting_scorecard=final_batting_scorecard.dropna(axis=1, how='all')

#Rename Columns
final_batting_scorecard=final_batting_scorecard.rename(columns={'BATSMEN': 'Player_Name'})
final_batting_scorecard=final_batting_scorecard.rename(columns={'Unnamed: 1': 'Dismissal_Detail'})

#Dervie Not out details
final_batting_scorecard.loc[final_batting_scorecard['Dismissal_Detail']=='not out', 'is_out'] ='N'
final_batting_scorecard.loc[final_batting_scorecard['Dismissal_Detail']=='retired hurt', 'is_out'] ='N'
final_batting_scorecard.loc[final_batting_scorecard['Dismissal_Detail']!='not out', 'is_out'] ='Y'


#Dervie batting pos
final_batting_scorecard=final_batting_scorecard.reset_index()
final_batting_scorecard['batting_pos'] = final_batting_scorecard.index

#Clean Player Name
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.replace('\(c\)','')
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.strip()
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.replace('†','')
final_batting_scorecard['Player_Name'] = final_batting_scorecard['Player_Name'].str.strip()

#Update data types
final_batting_scorecard['R'] = final_batting_scorecard['R'].astype(int)
final_batting_scorecard['B'] = final_batting_scorecard['B'].astype(int)
final_batting_scorecard['4s'] = final_batting_scorecard['4s'].astype(int)
final_batting_scorecard['6s'] = final_batting_scorecard['6s'].astype(int)
final_batting_scorecard['SR'] = final_batting_scorecard['SR'].astype(float)
final_batting_scorecard

,index,Player_Name,Dismissal_Detail,R,B,4s,6s,SR,Team,Team_Field,is_out,batting_pos
0,0,RG Sharma,c Curran b Chawla,12,10,2,0,120.00,MI,CSK,Y,0
1,2,Q de Kock,c Watson b Curran,33,20,5,0,165.00,MI,CSK,Y,1
2,4,SA Yadav,c Curran b Chahar,17,16,2,0,106.25,MI,CSK,Y,2
3,6,SS Tiwary,c du Plessis b Jadeja,42,31,3,1,135.48,MI,CSK,Y,3
4,8,HH Pandya,c du Plessis b Jadeja,14,10,0,2,140.00,MI,CSK,Y,4
5,10,KA Pollard,c †Dhoni b Ngidi,18,14,1,1,128.57,MI,CSK,Y,5
6,12,KH Pandya,c †Dhoni b Ngidi,3,3,0,0,100.00,MI,CSK,Y,6
7,14,JL Pattinson,c du Plessis b Ngidi,11,8,2,0,137.50,MI,CSK,Y,7
8,16,RD Chahar,not out,2,4,0,0,50.00,MI,CSK,N,8
9,18,TA Boult,b Chahar,0,1,0,0,0.00,MI,CSK,Y,9


In [4]:
import operator

final_batting_scorecard['4s_points']=final_batting_scorecard['4s']*5
final_batting_scorecard['6s_points']=final_batting_scorecard['6s']*10
final_batting_scorecard['runs_points']=0
final_batting_scorecard.loc[final_batting_scorecard['R'] >=10, 'runs_points'] =final_batting_scorecard['R']*1

final_batting_scorecard['runs_bonus_points']=0
final_batting_scorecard.loc[final_batting_scorecard['R'] >=50, 'runs_bonus_points'] = 25
final_batting_scorecard.loc[final_batting_scorecard['R'] >=100, 'runs_bonus_points'] = 75
final_batting_scorecard.loc[operator.and_(final_batting_scorecard['R']==0, final_batting_scorecard['is_out']!='N'), 'runs_bonus_points'] =-20
final_batting_scorecard['sr_points']=0
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=0 ),final_batting_scorecard['SR']<50), 'sr_points'] =-30
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=50 ),final_batting_scorecard['SR']<75), 'sr_points'] =-20
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=75 ),final_batting_scorecard['SR']<100), 'sr_points'] =-10
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=100), final_batting_scorecard['SR']<125), 'sr_points'] =0
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=125), final_batting_scorecard['SR']<175), 'sr_points'] =10
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=175), final_batting_scorecard['SR']<250), 'sr_points'] =20
final_batting_scorecard.loc[operator.and_(operator.and_(operator.or_(final_batting_scorecard['R'] >=10, final_batting_scorecard['B'] >=10),final_batting_scorecard['SR']>=250), final_batting_scorecard['SR']<1000), 'sr_points'] =30
final_batting_scorecard['total_batting_points']=final_batting_scorecard['4s_points']+final_batting_scorecard['6s_points']+final_batting_scorecard['runs_points']+final_batting_scorecard['sr_points']+final_batting_scorecard['runs_bonus_points']
final_batting_scorecard=final_batting_scorecard.sort_values(by=['Player_Name'])
final_batting_scorecard=final_batting_scorecard.reset_index()
del final_batting_scorecard['index']
del final_batting_scorecard['level_0']
final_batting_scorecard

,Player_Name,Dismissal_Detail,R,B,4s,6s,SR,Team,Team_Field,is_out,batting_pos,4s_points,6s_points,runs_points,runs_bonus_points,sr_points,total_batting_points
0,AT Rayudu,c & b Chahar,71,48,6,3,147.91,CSK,MI,Y,14,30,30,71,25,10,166
1,F du Plessis,not out,58,44,6,0,131.81,CSK,MI,N,13,30,0,58,25,10,123
2,HH Pandya,c du Plessis b Jadeja,14,10,0,2,140.00,MI,CSK,Y,4,0,20,14,0,10,44
3,JJ Bumrah,not out,5,3,0,0,166.66,MI,CSK,N,10,0,0,0,0,0,0
4,JL Pattinson,c du Plessis b Ngidi,11,8,2,0,137.50,MI,CSK,Y,7,10,0,11,0,10,31
5,KA Pollard,c †Dhoni b Ngidi,18,14,1,1,128.57,MI,CSK,Y,5,5,10,18,0,10,43
6,KH Pandya,c †Dhoni b Ngidi,3,3,0,0,100.00,MI,CSK,Y,6,0,0,0,0,0,0
7,M Vijay,lbw b Pattinson,1,7,0,0,14.28,CSK,MI,Y,11,0,0,0,0,0,0
8,MS Dhoni,not out,0,2,0,0,0.00,CSK,MI,N,17,0,0,0,0,0,0
9,Q de Kock,c Watson b Curran,33,20,5,0,165.00,MI,CSK,Y,1,25,0,33,0,10,68


In [5]:


for index, row in final_batting_scorecard.iterrows():
    first, *middle, last = str(row["Player_Name"]).split()
    player_name_short=first[0].lower()+"%"+last.lower()
    all_queries_list.append("Update ipl_match_stats_new set batting_pos="+str(row["batting_pos"])+
           " where lower(player_name) like '"+player_name_short+"%' and team_name='"+row["Team"]+
           "' and match_no="+str(match_no)+";")

for index, row in final_batting_scorecard.iterrows():
    all_queries_list.append("Update ipl_match_stats_new set runs_scored="+str(row["R"])+
                            ",balls_faced="+str(row["B"])+
                            ",fours_scored="+str(row["4s"])+
                            ",sixes_scored="+str(row["6s"])+
                            ",is_out='"+str(row["is_out"])+
                            "' where batting_pos="+str(row["batting_pos"])+" and match_no="+str(match_no)+";")
    


In [6]:
#Player_Name	Team_BOWL	O	M	R	W	Econ	0s	4s	6s	WD	NB	wicket_points	wicket_bonus_points	maiden_points	dot_ball_points	econ_points	total_bowling_points
#extract 1st innings bowling score
dfbowl1 = pd.read_html(match_link,skiprows=0)[1]
dfbowl1['Team_BOWL']=innings[1]
#extract 2nd innings bowling score
dfbowl2 = pd.read_html(match_link,skiprows=0)[3]
dfbowl2['Team_BOWL']=innings[0]

#combine bowling scores
final_bowling_scorecard=pd.concat([dfbowl1, dfbowl2])

#Remove unncessary rows
final_bowling_scorecard=final_bowling_scorecard[final_bowling_scorecard.BOWLING != 'TOTAL']

#Rename columns
final_bowling_scorecard=final_bowling_scorecard.rename(columns={'BOWLING': 'Player_Name'})
final_bowling_scorecard=final_bowling_scorecard.rename(columns={'ECON': 'Econ'})


#Update data types
final_bowling_scorecard['O'] = final_bowling_scorecard['O'].astype(float)
final_bowling_scorecard['M'] = final_bowling_scorecard['M'].astype(int)
final_bowling_scorecard['R'] = final_bowling_scorecard['R'].astype(int)
final_bowling_scorecard['W'] = final_bowling_scorecard['W'].astype(int)
final_bowling_scorecard['Econ'] = final_bowling_scorecard['Econ'].astype(float)
final_bowling_scorecard['0s'] = final_bowling_scorecard['0s'].astype(int)
final_bowling_scorecard['4s'] = final_bowling_scorecard['4s'].astype(int)
final_bowling_scorecard['6s'] = final_bowling_scorecard['6s'].astype(int)
final_bowling_scorecard['WD'] = final_bowling_scorecard['WD'].astype(int)
final_bowling_scorecard['NB'] = final_bowling_scorecard['NB'].astype(int)

In [7]:
final_bowling_scorecard['wicket_points']=final_bowling_scorecard['W']*25
final_bowling_scorecard['wicket_bonus_points']=0
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=3, 'wicket_bonus_points'] = 25
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=4,'wicket_bonus_points'] = 50
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=5,'wicket_bonus_points'] = 75
final_bowling_scorecard.loc[final_bowling_scorecard['W'] >=5,'wicket_bonus_points'] = 75
final_bowling_scorecard['maiden_points']=final_bowling_scorecard['M']*25
final_bowling_scorecard['dot_ball_points']=0
final_bowling_scorecard['dot_ball_points']=4*(final_bowling_scorecard['0s']-final_bowling_scorecard['4s']-1.5*final_bowling_scorecard['6s'])

final_bowling_scorecard['econ_points']=0
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>0 ),final_bowling_scorecard['Econ']<=4), 'econ_points'] =25
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>4 ),final_bowling_scorecard['Econ']<=6), 'econ_points'] =15
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>6 ),final_bowling_scorecard['Econ']<=8), 'econ_points'] =10
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>8 ),final_bowling_scorecard['Econ']<=10), 'econ_points'] =0
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>=10 ),final_bowling_scorecard['Econ']<12), 'econ_points'] =-10
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>=12 ),final_bowling_scorecard['Econ']<14), 'econ_points'] =-15
final_bowling_scorecard.loc[operator.and_(operator.and_(final_bowling_scorecard['O'] >=2,final_bowling_scorecard['Econ']>=14 ),final_bowling_scorecard['Econ']<100), 'econ_points'] =-25
final_bowling_scorecard['total_bowling_points']=final_bowling_scorecard['maiden_points']+final_bowling_scorecard['wicket_points']+final_bowling_scorecard['wicket_bonus_points']+final_bowling_scorecard['dot_ball_points']+final_bowling_scorecard['econ_points']
final_bowling_scorecard=final_bowling_scorecard.sort_values(by=['Player_Name'])
final_bowling_scorecard=final_bowling_scorecard.reset_index()
final_bowling_scorecard['bowling_pos'] = final_bowling_scorecard.index
del final_bowling_scorecard['index']
final_bowling_scorecard

,Player_Name,O,M,R,W,Econ,0s,4s,6s,WD,NB,Team_BOWL,wicket_points,wicket_bonus_points,maiden_points,dot_ball_points,econ_points,total_bowling_points,bowling_pos
0,DL Chahar,4.0,0,32,2,8.00,7,3,0,1,0,CSK,50,0,0,16.0,10,76.0,0
1,JJ Bumrah,4.0,0,43,1,10.75,10,4,2,1,1,MI,25,0,0,12.0,-10,27.0,1
2,JL Pattinson,4.0,0,27,1,6.75,10,3,0,1,0,MI,25,0,0,28.0,10,63.0,2
3,KH Pandya,4.0,0,37,1,9.25,2,1,2,1,0,MI,25,0,0,-8.0,0,17.0,3
4,L Ngidi,4.0,0,38,3,9.50,8,6,0,2,0,CSK,75,25,0,8.0,0,108.0,4
5,PP Chawla,4.0,0,21,1,5.25,11,0,1,1,0,CSK,25,0,0,38.0,15,78.0,5
6,RA Jadeja,4.0,0,42,2,10.50,5,2,3,0,0,CSK,50,0,0,-6.0,-10,34.0,6
7,RD Chahar,4.0,0,36,1,9.00,2,3,1,0,0,MI,25,0,0,-10.0,0,15.0,7
8,SM Curran,4.0,0,28,1,7.00,9,4,0,0,0,CSK,25,0,0,20.0,10,55.0,8
9,TA Boult,3.2,0,23,1,6.90,12,5,0,0,0,MI,25,0,0,28.0,10,63.0,9


In [8]:
for index, row in final_bowling_scorecard.iterrows():
    first, *middle, last = str(row["Player_Name"]).split()
    player_name_short=first[0].lower()+"%"+last.lower()
    all_queries_list.append("Update ipl_match_stats_new set bowling_pos="+str(row["bowling_pos"])+
           " where lower(player_name) like '"+player_name_short+"%' and team_name='"+row["Team_BOWL"]+
           "' and match_no="+str(match_no)+";")


for index, row in final_bowling_scorecard.iterrows():
    all_queries_list.append("Update ipl_match_stats_new set overs_bowled="+str(row["O"])+
           ",maidens="+str(row["M"])+
           ",runs_given="+str(row["R"])+
           ",wickets="+str(row["W"])+
           ",economy="+str(row["Econ"])+
           ",dots="+str(row["0s"])+
           ",fours="+str(row["4s"])+
           ",sixes="+str(row["6s"])+
           " where bowling_pos="+str(row["bowling_pos"])+" and match_no="+str(match_no)+";")


In [9]:
#fielding calculation
catches_list=[]
direct_run_out_list=[]
part_of_run_out_list=[]
stumpings_list=[]

#generate a smaller fielding input data frame from final batting scorecard
fielding_input=pd.DataFrame() 
fielding_input['Dismissal_Detail']=final_batting_scorecard['Dismissal_Detail']
fielding_input['Team_Field']=final_batting_scorecard['Team_Field']

#Generate 4 types of fielding scorecards (catches_list, direct run out, part of run out, stumpings)
for index, row in fielding_input.iterrows():
    test_string=row["Dismissal_Detail"]
    test_string=test_string.replace("retired hurt","not out")
    ##Just for the 31st match
    test_string=test_string.replace("run out (Pandya)","run out (Hardik Pandya)")

    #team_commentary.append(test_string)
    st_pos=test_string.find('st ')

    if(("c & b") in test_string):
        test_string=test_string.replace("c & b","c")+test_string.replace("c & b"," b")
    if test_string[0]=="c" and test_string[1]==" ":
        test_string=test_string[2::]
    elif test_string[0]=="c" and test_string[1]!=" ":
        test_string=test_string[1::]
    bpos=test_string.rfind(' b ')
    if(bpos!=-1):
        test_string=test_string[0:bpos].strip()

    run_out_pos=test_string.find('run out (')

    if(test_string!='' and test_string!='lbw'  and test_string!='not out'):
        if(st_pos!=0) and (run_out_pos!=0) and not ("b ")in test_string:
            test_string=test_string.replace("†","").strip()
            catches_list.append(test_string+":::"+row['Team_Field'])
    if(run_out_pos==0 and '/' not in test_string):
        test_string=test_string.replace("run out (","")
        test_string=test_string.replace(")","")
        test_string=test_string.replace("†","").strip()
        direct_run_out=test_string
        if(direct_run_out_list!=''):
            direct_run_out_list.append(direct_run_out+row['Team_Field'])
    if(run_out_pos==0 and '/' in test_string):
        run_out_pos=test_string.find('run out (')
        if(run_out_pos==0):
            test_string=test_string.replace("run out (","")
            test_string=test_string.replace(")","")
        test_string=test_string.replace("†","").strip()
        part_of_run_out_guys=test_string.split("/")
        for part_of_run_out in part_of_run_out_guys:
            #print(part_of_run_out)
            if(part_of_run_out!='/'):
                part_of_run_out_list.append(part_of_run_out+row['Team_Field'])
                #print(part_of_run_out_list)
    if(st_pos==0):
        test_string=test_string[3::]
        test_string=test_string.replace("†","").strip()
        stumpings_list.append(test_string+row['Team_Field'])

In [10]:
#get final catches scorecard
if len(catches_list) > 0:
    catch_scorecard=pd.DataFrame(catches_list)
    catch_scorecard['Player_Name']= catch_scorecard[0].map(lambda v: v.split(':::')[0])
    catch_scorecard['Team_Field']= catch_scorecard[0].map(lambda v: v.split(':::')[1])
    del catch_scorecard[0]
    catch_scorecard=catch_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Catches')
else:
    catch_scorecard=pd.DataFrame()


#get direct run out scorecard
if len(direct_run_out_list)>0:
    direct_run_out_scorecard=pd.DataFrame(direct_run_out_list)
    direct_run_out_scorecard['Player_Name']= final_runout_direct_scorecard[0].map(lambda v: v.split(':::')[0])
    direct_run_out_scorecard['Team_Field']= final_runout_direct_scorecard[0].map(lambda v: v.split(':::')[1])
    del direct_run_out_scorecard[0]
    direct_run_out_scorecard=direct_run_out_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Direct_Run_Out')
else:
    direct_run_out_scorecard=pd.DataFrame()


#get part of run out scorecard
if len(part_of_run_out_list)>0:
    part_of_run_out_scorecard=pd.DataFrame(part_of_run_out_list)
    part_of_run_out_scorecard['Player_Name']= final_runout_direct_scorecard[0].map(lambda v: v.split(':::')[0])
    part_of_run_out_scorecard['Team_Field']= final_runout_direct_scorecard[0].map(lambda v: v.split(':::')[1])
    del part_of_run_out_scorecard[0]
    part_of_run_out_scorecard=part_of_run_out_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Part_Of_Run_Out')
else:
    part_of_run_out_scorecard=pd.DataFrame()

#get stumpings scorecard
if len(stumpings_list)>0:
    stumpings_scorecard=pd.DataFrame(stumpings_list)
    stumpings_scorecard['Player_Name']= final_runout_direct_scorecard[0].map(lambda v: v.split(':::')[0])
    stumpings_scorecard['Team_Field']= final_runout_direct_scorecard[0].map(lambda v: v.split(':::')[1])
    del stumpings_scorecard[0]
    stumpings_scorecard=stumpings_scorecard.groupby(['Player_Name','Team_Field']).size().reset_index(name='Stumpings')
else:
    stumpings_scorecard=pd.DataFrame()

In [11]:
catch_scorecard

,Player_Name,Team_Field,Catches
0,Chahar,MI,1
1,Curran,CSK,2
2,Dhoni,CSK,2
3,Pattinson,MI,1
4,Watson,CSK,1
5,du Plessis,CSK,3


In [12]:
direct_run_out_scorecard

""


In [13]:
part_of_run_out_scorecard

""


In [14]:
stumpings_scorecard

""


In [15]:
def extract_player_name(player_name):
    player_name_list=player_name.split()
    player_first_name_part=""
    player_name_final="%"+player_name_list[-1]+"%"
    if len(player_name_list)>1:
        player_first_name_part="%"+player_name_list[0][0]+""
        player_name_final=player_first_name_part+player_name_final
        # only problem is Hardik and Krunal Pandya
        if("%h%pandya%" in player_name_final.lower()):
            player_name_final="h%pandya%"
        elif("%k%pandya%" in player_name_final.lower()):
            player_name_final="k%pandya%"
    player_name_final=player_name_final.lower().replace("sub(","").replace("(","").replace(")","")
    return player_name_final

In [16]:
for index, row in catch_scorecard.iterrows():
    player_name_final=extract_player_name(str(row["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set catches="+str(row["Catches"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+row["Team_Field"]+
           "' and match_no="+str(match_no)+";")

In [17]:
for index, row in direct_run_out_scorecard.iterrows():
    player_name_final=extract_player_name(str(row["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set direct_run_out="+str(row["Direct_Run_Out"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+row["Team"]+
           "' and match_no="+str(match_no)+";")

In [18]:
for index, row in part_of_run_out_scorecard.iterrows():
    player_name_final=extract_player_name(str(row["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set part_of_run_out="+str(row["Part_Of_Run_Out"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+row["Team"]+
           "' and match_no="+str(match_no)+";")

In [19]:
for indexs, rows in stumpings_scorecard.iterrows():
    player_name_final=extract_player_name(str(rows["Player_Name"]))
    all_queries_list.append("Update ipl_match_stats_new set stumpings="+str(rows["Stumpings"])+
           " where lower(player_name) like '"+player_name_final+"' and team_name='"+rows["Team"]+
           "' and match_no="+str(match_no)+";")

In [20]:
#for duck_out_player in duck_out_in0ball_players_list:
    #if(duck_out_player!=""):
        #all_queries_list.append("update ipl_match_stats_new  set is_out='Y' where match_no="+str(match_no)+"  and balls_faced =0 and lower(player_name) like lower('"+duck_out_player+"');")


In [21]:
#all_queries_list.append("update ipl_match_stats_new  set is_out='Y' where match_no="+str(match_no)+"  and balls_faced >0;")
all_queries_list.append("update ipl_match_stats_new  set strike_rate=ROUND((runs_scored*100/balls_faced),2) where match_no="+str(match_no)+" and balls_faced>0;")
all_queries_list.append("update ipl_match_stats_new  set economy=ROUND((runs_given/(floor(overs_bowled)+(overs_bowled-floor(overs_bowled))/6*10)),2) where match_no="+str(match_no)+"  and overs_bowled>0;")

all_queries_list.append("\n------------Manually update if any update has zero records updated add replace based on player name \n\n\n\n")
all_queries_list.append("\nupdate ipl_match_stats_new  set is_part_of_winning_team='Y' where match_no="+str(match_no)+"  and team_name='"+winning_team+"';") 
all_queries_list.append("update ipl_match_stats_new  set is_part_of_winning_team='N' where match_no="+str(match_no)+"  and team_name<>'"+winning_team+"';") 
all_queries_list.append("update ipl_match_stats_new  set is_mom='N' where match_no="+str(match_no)+" and player_name not like '%"+man_of_the_match+"%';") 
all_queries_list.append("update ipl_match_stats_new  set is_mom='Y' where match_no="+str(match_no)+" and player_name like '%"+man_of_the_match+"%';") 

#for not_out_player in not_out_in_0_players_list:
    #if(not_out_player!=""):
        #all_queries_list.append("update ipl_match_stats_new  set is_out='N' where match_no="+str(match_no)+"  and balls_faced >0 and lower(player_name) like lower('"+not_out_player+"');")
        


In [22]:
import numpy as np
import re
result = pd.concat([final_batting_scorecard.set_index('Player_Name'), final_bowling_scorecard.set_index('Player_Name')], axis=1,sort=True)
result=result.fillna(0)
result2=finaldata=pd.DataFrame()
result2['total_batting_points']=result['total_batting_points']
result2['total_bowling_points']=result['total_bowling_points']
result2['Team_BAT']=result['Team']
result2['Team_BOWL']=result['Team_BOWL']


conditions = [
    result2['Team_BAT']=='0', 
    result2['Team_BAT']!='0']

choices = [result2['Team_BOWL'], result2['Team_BAT']]

result2['Team'] = np.select(conditions, choices)


result2.loc[result2['Team']=='0','Team'] =result['Team_BOWL']

result2['total_bat_bowl_points']=result2['total_batting_points']+result2['total_bowling_points']
result2['Temp_Player_Name'] = result2.index
result2=result2.reset_index()
del result2['index']
result2.insert(0, 'Player_Name', result2['Temp_Player_Name'])
del result2['Temp_Player_Name']

result3=finaldata=pd.DataFrame()

result2_rows_list=[]

for index, row in result2.iterrows():
    if(row["Team_BAT"]==0):
        row["Team"]=row["Team_BOWL"]
    if(row["Team_BOWL"]==0):
        row["Team"]=row["Team_BAT"]
    row["catches_points"]=0
    row["part_of_run_out_points"]=0
    row["direct_run_out_points"]=0
    row["stumpings_points"]=0

    player_name=row["Player_Name"]
    for indexc, rowc in catch_scorecard.iterrows():
        pattern=extract_player_name(rowc["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                row["catches_points"]=rowc["Catches"]*5
                break
        except:
            pass
    
    for indexdr, rowdr in direct_run_out_scorecard.iterrows():
        pattern=extract_player_name(rowdr["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        #print("pattern:::"+pattern)
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                #print(player_name)
                row["direct_run_out_points"]=rowdr["Direct_Run_Out"]*15
                break
        except:
            pass
    
    
    for indexpr, rowpr in part_of_run_out_scorecard.iterrows():
        pattern=extract_player_name(rowpr["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                row["part_of_run_out_points"]=rowpr["Part_Of_Run_Out"]*5
                break
        except:
            pass
    
    
    for indexs, rows in stumpings_scorecard.iterrows():
        pattern=extract_player_name(rows["Player_Name"]).replace("%",".").replace("h.pandya.","hh.pandya.").replace("k.pandya.","kh.pandya.")
        compiled = re.compile(pattern)
        ms = compiled.search(player_name.lower()+" ")
        try:
            if(ms.string!=''):
                row["stumpings_points"]=rows["Stumpings"]*10
                break
        except:
            pass
    
    row["Total_Batting_Points"]=row["total_batting_points"]
    row["Total_Bowling_Points"]=row["total_bowling_points"]

    row["Total_Fielding_Points"]=row["stumpings_points"]+row["part_of_run_out_points"]+row["direct_run_out_points"]+row["catches_points"]
    row["Total_Points"]=row["total_batting_points"]+row["total_bowling_points"]+row["Total_Fielding_Points"]
    
    result2_rows_list.append(row)
result3 = pd.DataFrame(result2_rows_list,)
del result3['Team_BAT']
del result3['Team_BOWL']
del result3['stumpings_points']
del result3['part_of_run_out_points']
del result3['direct_run_out_points']
del result3['catches_points']
del result3['total_bat_bowl_points']
del result3['total_batting_points']
del result3['total_bowling_points']


result3

,Player_Name,Team,Total_Batting_Points,Total_Bowling_Points,Total_Fielding_Points,Total_Points
0,AT Rayudu,CSK,166.0,0.0,0,166.0
1,DL Chahar,CSK,0.0,76.0,5,81.0
2,F du Plessis,CSK,123.0,0.0,0,123.0
3,HH Pandya,MI,44.0,0.0,0,44.0
4,JJ Bumrah,MI,0.0,27.0,0,27.0
5,JL Pattinson,MI,31.0,63.0,5,99.0
6,KA Pollard,MI,43.0,0.0,0,43.0
7,KH Pandya,MI,0.0,17.0,0,17.0
8,L Ngidi,CSK,0.0,108.0,0,108.0
9,M Vijay,CSK,0.0,0.0,0,0.0


In [23]:
teams_list=teams.split("\t")
teams_list_len=len(teams_list)-1
person_count=int(teams_list_len/17)

match_no_list=[]
match_title_list=[]
person_name_list=[]
captain_list=[]
mom_list=[]
winner_list=[]
player_list=[]

for i in range(0, person_count):
    for j in range(0, 11):
        match_no_list.append(teams_list[(i*17)+0])
        match_title_list.append(teams_list[(i*17)+1])
        person_name_list.append(teams_list[(i*17)+2])
        captain_list.append(teams_list[(i*17)+14])
        mom_list.append(teams_list[(i*17)+15])
        winner_list.append(teams_list[(i*17)+16])

    player_list.append(teams_list[(i*17)+3])
    player_list.append(teams_list[(i*17)+4])
    player_list.append(teams_list[(i*17)+5])
    player_list.append(teams_list[(i*17)+6])
    player_list.append(teams_list[(i*17)+7])
    player_list.append(teams_list[(i*17)+8])
    player_list.append(teams_list[(i*17)+9])
    player_list.append(teams_list[(i*17)+10])
    player_list.append(teams_list[(i*17)+11])
    player_list.append(teams_list[(i*17)+12])
    player_list.append(teams_list[(i*17)+13])
team_selected_list=pd.DataFrame({'Match_No' : match_no_list,'Match_Title':match_title_list,
         'Person_Name' : person_name_list,
         'Player_Name': player_list,
         'Captain':captain_list,
         'Man_of_the_Match':mom_list,
         'winner':winner_list
         }, columns=['Match_No','Match_Title','Person_Name','Player_Name','Captain','Man_of_the_Match','winner'])
team_selected_list['Is_Captain']="N"
team_selected_list['Is_MoM']="N"
team_selected_list.loc[team_selected_list['Player_Name']==team_selected_list["Captain"],'Is_Captain'] = "Y"
team_selected_list.loc[team_selected_list['Player_Name']==team_selected_list["Man_of_the_Match"],'Is_MoM'] = "Y"
team_selected_list["points"]=0
del team_selected_list["Captain"]
del team_selected_list["Man_of_the_Match"]
#team_selected_list["Winner"]=team_selected_list["winner"]
#del team_selected_list["winner"]


In [24]:
for ii in range (1,3):
    teams_selected_fuzzy_list=[]
    for index1, row1 in team_selected_list.iterrows():
        prev_match_score=0
        my_player_name=""
        #print ("player_name1:"+player_name1)
        for index2, row2 in result3.iterrows():
            name_list_2 = row2["Player_Name"].split();
            if(row1["Player_Name"]=="Krunal Pandya"):
                row1["Player_Name"]="KH Pandya"
            elif(row1["Player_Name"]=="Hardik Pandya"):
                row1["Player_Name"]="HH Pandya"
            elif(row1["Player_Name"]=="Mujeeb Zadran"):
                row1["Player_Name"]="Mujeeb Ur Rahman"
            elif(row1["Player_Name"]=="Dinesh Karthik"):
                row1["Player_Name"]="KD Karthik"
            elif(row1["Player_Name"]=="Rashid Khan Arman"):
                row1["Player_Name"]="Rashid Khan"
            elif(row1["Player_Name"]=="Gowtham Krishnappa"):
                row1["Player_Name"]="Krishnappa Gowtham"
            
            name_list_1 = row1["Player_Name"].split();
            player_name1=name_list_1[0][0]+name_list_1[-1]

                
            player_name2=name_list_2[0][0]+name_list_2[-1]
            
            #if(player_name2=="CGayle"):
                #print(player_name2+"   "+player_name1)
            
    
    
            if(player_name2==player_name1):
                #print("****"+row2["Player_Name"]+"  "+player_name2)
                #if(player_name2=="KWilliamson"):
                    #print(str(row2["Total_Points"])+"  "+row2["Player_Name"])
                #row1["Old_Player_Name"]=row1["Player_Name"]
                player_name_new=row2["Player_Name"]
                row1["Player_Name"]=player_name_new
                row1["points"]=int(row2["Total_Points"])
                if(row1["Is_Captain"]=='Y'):
                    row1["points"]=int(row1["points"]*2)
            row1["Player_Name"]=row1["Player_Name"].replace("✈","")
        teams_selected_fuzzy_list.append(row1)
    
    teams_selected_fuzzy = pd.DataFrame(teams_selected_fuzzy_list)
    
    
    #del(teams_selected_fuzzy["Old_Player_Name"])
    del(teams_selected_fuzzy["Is_MoM"])
    del(teams_selected_fuzzy["Is_Captain"])
    
    
    
    teams_selected_fuzzy["points_1"]=teams_selected_fuzzy["points"]*-1
    teams_selected_fuzzy=teams_selected_fuzzy.sort_values(by=['Person_Name','points_1'])
    del(teams_selected_fuzzy["points_1"])
    teams_selected_fuzzy_pd=pd.DataFrame({'sum' : teams_selected_fuzzy.groupby( ['Person_Name'] )["points"].sum()}).reset_index()
    teams_selected_fuzzy_pd
    
    teams_selected_fuzzy_pd2=pd.DataFrame({'winner' : teams_selected_fuzzy.groupby( ['Person_Name'] )["winner"].max()}).reset_index()
    teams_selected_fuzzy_pd2["winner_points"]= -50
    teams_selected_fuzzy_pd2.loc[teams_selected_fuzzy_pd2["winner"]==winning_team, 'winner_points'] = 50
    
    teams_selected_fuzzy_pd3=pd.DataFrame()
    
    teams_selected_fuzzy_pd3 = pd.concat([teams_selected_fuzzy_pd.set_index('Person_Name'), teams_selected_fuzzy_pd2.set_index('Person_Name')], axis=1)
    del(teams_selected_fuzzy_pd3["winner"])
    teams_selected_fuzzy_pd3["total_points"]=teams_selected_fuzzy_pd3["sum"]+teams_selected_fuzzy_pd3["winner_points"]
    del(teams_selected_fuzzy_pd3["sum"])
    del(teams_selected_fuzzy_pd3["winner_points"])
    print("")

In [25]:
print("Player Points")
teams_selected_fuzzy=teams_selected_fuzzy.sort_values(by=['Person_Name'],ascending=True)
teams_selected_fuzzy

Player Points


,Match_No,Match_Title,Person_Name,Player_Name,winner,points
66,1,MI vs CSK,Aditya,F du Plessis,CSK,123
75,1,MI vs CSK,Aditya,DL Chahar,CSK,81
70,1,MI vs CSK,Aditya,RA Jadeja,CSK,74
71,1,MI vs CSK,Aditya,HH Pandya,CSK,44
76,1,MI vs CSK,Aditya,TA Boult,CSK,43
68,1,MI vs CSK,Aditya,SA Yadav,CSK,27
74,1,MI vs CSK,Aditya,JJ Bumrah,CSK,27
69,1,MI vs CSK,Aditya,RG Sharma,CSK,22
72,1,MI vs CSK,Aditya,MS Dhoni,CSK,20
67,1,MI vs CSK,Aditya,SR Watson,CSK,10


In [26]:
print("Player Points#Summary")
teams_selected_fuzzy_pd=teams_selected_fuzzy_pd.sort_values(by=['sum'],ascending=False)
teams_selected_fuzzy_pd

Player Points#Summary


,Person_Name,sum
3,Govardhan,713
8,Suresh,633
2,Deepak,616
5,Mitesh,612
6,Shriyansh,547
4,Kiran,487
1,Avani,486
0,Aditya,471
7,Sowmya,284


In [27]:
print("Winner Team Points")
teams_selected_fuzzy_pd2

Winner Team Points


,Person_Name,winner,winner_points
0,Aditya,CSK,50
1,Avani,MI,-50
2,Deepak,MI,-50
3,Govardhan,MI,-50
4,Kiran,MI,-50
5,Mitesh,CSK,50
6,Shriyansh,CSK,50
7,Sowmya,MI,-50
8,Suresh,MI,-50


In [28]:
print("Total Points")
teams_selected_fuzzy_pd3
teams_selected_fuzzy_pd3=teams_selected_fuzzy_pd3.sort_values(by=['total_points'],ascending=False)
teams_selected_fuzzy_pd3

Total Points


,total_points
Person_Name,
Govardhan,663
Mitesh,662
Shriyansh,597
Suresh,583
Deepak,566
Aditya,521
Kiran,437
Avani,436
Sowmya,234


In [30]:
all_queries_str="\n".join(all_queries_list)

all_queries_str=all_queries_str.replace("k%karthik","d%karthik")
all_queries_str=all_queries_str.replace("k%gowtham","%gowtham%k%");
all_queries_str=all_queries_str.replace("lower(player_name) like 'm%rahman%' and team_name='KXIP'","lower(player_name) like 'mujeeb%' and team_name='KXIP'");
all_queries_str=all_queries_str.replace("klaasen%","kla%sen%");
all_queries_str=all_queries_str.replace("k%sharma%","karn%sharma%");
all_queries_str=all_queries_str.replace("m%krishna%","p%krishna%");
all_queries_str=all_queries_str.replace("scantlebury-searles","searles");
all_queries_str=all_queries_str.replace("like '%sharma%' and team_name='CSK'","like 'karn%sharma%' and team_name='CSK'")
all_queries_str=all_queries_str.replace("ipl_match_stats_new","ipl_2020_match_stats")

print ("--Update queries below:\n"+all_queries_str)



--Update queries below:
Update ipl_2020_match_stats set batting_pos=14 where lower(player_name) like 'a%rayudu%' and team_name='CSK' and match_no=1;
Update ipl_2020_match_stats set batting_pos=13 where lower(player_name) like 'f%plessis%' and team_name='CSK' and match_no=1;
Update ipl_2020_match_stats set batting_pos=4 where lower(player_name) like 'h%pandya%' and team_name='MI' and match_no=1;
Update ipl_2020_match_stats set batting_pos=10 where lower(player_name) like 'j%bumrah%' and team_name='MI' and match_no=1;
Update ipl_2020_match_stats set batting_pos=7 where lower(player_name) like 'j%pattinson%' and team_name='MI' and match_no=1;
Update ipl_2020_match_stats set batting_pos=5 where lower(player_name) like 'k%pollard%' and team_name='MI' and match_no=1;
Update ipl_2020_match_stats set batting_pos=6 where lower(player_name) like 'k%pandya%' and team_name='MI' and match_no=1;
Update ipl_2020_match_stats set batting_pos=11 where lower(player_name) like 'm%vijay%' and team_name='CS